In [2]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import uncertainties
from uncertainties import ufloat, unumpy
from uncertainties.umath import log10
from scipy.odr import ODR, Model, RealData
from scipy.optimize import fsolve, curve_fit
from matplotlib.ticker import AutoMinorLocator, LogLocator
from scipy import constants

In [3]:
resistance = 52.1
dOmega = 0.005 * resistance + 0.1
dOmega

0.36050000000000004

## Propagation velocity

### Coax cable

In [4]:
dt = np.sqrt(2) * 10
T = ufloat(508*1e-9, dt*1e-9)
l = ufloat(50, 0.01)
(2*l/T) / 1e8 # in 10^8 m/s

1.9685039370078738+/-0.05480229933904268

### Delay cable

In [5]:
dt_delay = np.sqrt(2) * 50
T_delay = ufloat(1.78*1e-6, dt*1e-9)
l_delay = ufloat(0.455, 0.02)
(2*l_delay/T_delay) / 1e5 # in 10^8 m/s
# dt_delay / 1e3

5.1123595505617985+/-0.2283604168872041

In [12]:
v_p = (2*l/T)
[(v_p / ((i)*l)) / 1e6 for i in [4/3, 2, 4]]

[2.9527559055118116+/-0.08220132771559158,
 1.968503937007874+/-0.05480088514372772,
 0.984251968503937+/-0.02740044257186386]

## Speed of light

In [7]:
lengths = [3.913, 4.013, 4.113, 7.923, 8.023, 8.123, 11.715, 11.815, 11.915]
times = np.array([20, 24, 24, 56, 55, 59, 82, 84, 86]) * 1e-9

m = (5-10) / (lengths[-1]-lengths[0])
a = 10-m*lengths[0]

errTimes = np.array([np.sqrt(2) * abs(m*i+a)  for i in lengths]) * 1e-9
d = unumpy.uarray(lengths, np.full(len(lengths), np.sqrt(2) * 0.002))
t = unumpy.uarray(times, errTimes)

c = (2 * d / t) / 1e8
c

array([3.913+/-2.766910280439176, 3.3441666666666663+/-1.9582575460078342,
       3.4275+/-1.9944355246343737,
       2.829642857142857+/-0.5355436445467966,
       2.917454545454545+/-0.5575149750798377,
       2.7535593220338974+/-0.4863969555493667,
       2.857317073170732+/-0.25255293742392126,
       2.8130952380952374+/-0.23976487294133655,
       2.770930232558139+/-0.22783159576975812], dtype=object)

In [8]:
def weight_avg(data):
    weights = 1 / unumpy.std_devs(data)**2
    avg = np.sum(unumpy.nominal_values(data)*weights) / np.sum(weights)
    std_err = 1 / np.sqrt(np.sum(weights))
    return f'{avg} ± {std_err}'

In [9]:
weight_avg(c)

'2.820283511406525 ± 0.12508596633341607'

In [10]:
def dampening(U0, U):
    return 20 * log10(U0 / (U0 - U))

In [11]:
U0 = ufloat(5, 0.01)
dampening(U0, ufloat(0.5, 0.1))

0.9151498112135028+/-0.19302942048200264